In [52]:
%load_ext autoreload
%autoreload 2
import os
import torch
import json
import yaml
import tqdm
from baseline.models.AggConcatNet import AggConcatNet
from dataset.MurelNetDataset import MurelNetDataset
from baseline.models.AttentionNet import AttentionNet
from murel.models.MurelNet import MurelNet
from torch.utils.data import DataLoader
import transforms.transforms as trfm

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [53]:
def eval_model(model, BEST_MODEL_PATH, RESULTS_FILE_PATH):
    #Path to model weights with best performance on the validation dataset
    print('Processing %s' % BEST_MODEL_PATH)
    model.load_state_dict(torch.load(BEST_MODEL_PATH)['model'])
    model = model.cuda()
    model = model.eval()
    results = []
    with torch.no_grad():
        for data in tqdm.tqdm(test_loader):
            item = {\
                'question_ids': data['question_ids'].cuda(), \
                'object_features_list': data['object_features_list'].cuda(), \
                'bounding_boxes': data['bounding_boxes'].cuda(), \
                'question_lengths': data['question_lengths'].cuda(), \
            }
            inputs = item
            qids = data['question_unique_id']
            outputs = model(inputs)
            values, ans_indices = torch.max(outputs, dim=1)
            ans_indices = list(ans_indices)
            ans_indices = [tsr.item() for tsr in ans_indices]
            for qid, ans_idx in zip(qids, ans_indices):
                results.append({
                    'question_id': int(qid),
                    'answer': test_dataset.aid_to_ans[ans_idx]
                })
    print('Saving results to %s' % RESULTS_FILE_PATH)
    with open(RESULTS_FILE_PATH, 'w') as f:
        json.dump(results, f)
    print('Done saving to %s' % RESULTS_FILE_PATH)

In [54]:
DATA_DIR = "/auto/homes/bat34/VQA/"
ROOT_DIR = "/auto/homes/bat34/VQA_PartII/"
MODEL_NAME = 'baseline'
SPLIT = 'val'
CONFIG_PATH = os.path.join(ROOT_DIR, MODEL_NAME, 'scripts', 'attention_baseline' + '.yaml')
with open(CONFIG_PATH) as f:
        config = yaml.load(f)
        
train_dataset = MurelNetDataset(split='train', \
                        txt_enc=config['txt_enc'], \
                        bottom_up_features_dir=config['bottom_up_features_dir'], \
                        skipthoughts_dir=config['skipthoughts_dir'], \
                        processed_dir=config['processed_dir'], \
                        ROOT_DIR=ROOT_DIR, \
                        vqa_dir=config['vqa_dir'])

test_dataset = MurelNetDataset(split=SPLIT, \
                        txt_enc=config['txt_enc'], \
                        bottom_up_features_dir=config['bottom_up_features_dir'], \
                        skipthoughts_dir=config['skipthoughts_dir'], \
                        processed_dir=config['processed_dir'], \
                        ROOT_DIR=ROOT_DIR, \
                        vqa_dir=config['vqa_dir'])

print('{} Dataset Loaded.'.format(SPLIT))

test_loader = DataLoader(test_dataset, shuffle=False, \
                         num_workers=config['num_workers'], \
                         batch_size=config['batch_size'] // 4, \
                         collate_fn=test_dataset.collate_fn, \
)

print('{} DataLoader loaded.'.format(SPLIT))
    
word_vocabulary = [word for word, _ in train_dataset.word_to_wid.items()]
#model = MurelNet(config, word_vocabulary)
model = AttentionNet(config, word_vocabulary)

BEST_MODEL_NAME = 'concat_baseline_self_attention_block_txt_enc_BayesianUniSkip_batch_size_256_lr_0.0001_lr_decay_rate_0.25_hidden_list_[1600, 1600]_attention_fusion_type_block_final_fusion_type_block'
#BEST_MODEL_NAME = 'murel_reverse_question_object_txt_enc_BayesianUniSkip_batch_size_256_lr_5e-05_lr_decay_rate_0.25_unroll_steps_3_fusion_type_block'
#BEST_MODEL_NAME_TEMPL = 'agg_concat_{}_agg_type_{}_q_self_attention_{}' + \
                  #'_txt_enc_BayesianUniSkip_batch_size_256_lr_0.0001_fusion_type_concat_mlp' 


Processing....


/home/bat34/venv/lib/python3.5/site-packages/ipykernel_launcher.py:7: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  import sys
  1% (4785 of 443757) |                  | Elapsed Time: 0:00:00 ETA:   0:00:09

Tokenizing questions for train2014


100% (443757 of 443757) |################| Elapsed Time: 0:00:09 Time:  0:00:09
  2% (5427 of 214354) |                  | Elapsed Time: 0:00:00 ETA:   0:00:04

Tokenizing questions for val2014


...

No. of known words: 2752210, No. of unknown words : 1399, Percentage Loss of words: 0.050806051258548326%
Removing questions if they have infrequent answers


100% (443757 of 443757) |################| Elapsed Time: 0:00:01 Time:  0:00:01
  0% (978 of 413036) |                   | Elapsed Time: 0:00:00 ETA:   0:00:42

Adding weight list for id occurences..


100% (413036 of 413036) |################| Elapsed Time: 0:00:28 Time:  0:00:28


Saving processed datasets...
Finished processing annotations and questions.
val Dataset Loaded.
val DataLoader loaded.


In [55]:
iter_train = iter(train_dataset)

In [118]:
x = next(iter_train)

In [119]:
x

{'answer': 'motorcycle',
 'answer_id': tensor([72]),
 'bounding_boxes': tensor([[0.0325, 0.0000, 0.9984, 0.4117],
         [0.1439, 0.4076, 0.6005, 0.8460],
         [0.3979, 0.2317, 0.6060, 0.4161],
         [0.4424, 0.3983, 0.5688, 0.5770],
         [0.4667, 0.1624, 0.5533, 0.2470],
         [0.0020, 0.6714, 0.8951, 0.9983],
         [0.4685, 0.6419, 0.9753, 0.8799],
         [0.0262, 0.5853, 0.3379, 0.7079],
         [0.0541, 0.5846, 0.1421, 0.7016],
         [0.0140, 0.3480, 0.5992, 0.6229],
         [0.1463, 0.5546, 0.4091, 0.9530],
         [0.5583, 0.2612, 0.9523, 0.5546],
         [0.0000, 0.0750, 0.3784, 0.3785],
         [0.4652, 0.5463, 0.5774, 0.8386],
         [0.6953, 0.2544, 0.9869, 0.5920],
         [0.3452, 0.3473, 0.3861, 0.4051],
         [0.0000, 0.1244, 0.3087, 0.4421],
         [0.1935, 0.5260, 0.4488, 0.9465],
         [0.0511, 0.2036, 0.7672, 0.4328],
         [0.1337, 0.6075, 0.2064, 0.6792],
         [0.3379, 0.1655, 0.6295, 0.6923],
         [0.4713, 0.6221, 

In [122]:
y = torch.ones(1, 3000)

In [123]:
y.index_select(1, x['id_unique']) * x['id_weights']

tensor([[0.4444, 0.3333, 0.2222]])

In [125]:
x['id_unique'].is_leaf

True

In [5]:
#BEST_MODEL_NAME = 'murel_reverse_question_object_txt_enc_BayesianUniSkip_pooling_agg_min_pairwise_agg_max_batch_size_256_lr_5e-05_lr_decay_rate_0.25_unroll_steps_3_fusion_type_block'
#BEST_MODEL_NAME = 'murel_reverse_question_object_txt_enc_BayesianUniSkip_batch_size_256_lr_5e-05_lr_decay_rate_0.25_unroll_steps_3_fusion_type_block'
BEST_MODEL_PATH = os.path.join(ROOT_DIR, MODEL_NAME, 'trained_models', 'best_models', BEST_MODEL_NAME, 'best_model.pth')
RES_FILE_NAME = 'attention_block_{}-2015_results.json'.format(SPLIT)
RES_FILE_PATH = os.path.join(ROOT_DIR, MODEL_NAME, RES_FILE_NAME)
eval_model(model, BEST_MODEL_PATH, RES_FILE_PATH)

Processing /auto/homes/bat34/VQA_PartII/baseline/trained_models/best_models/concat_baseline_self_attention_block_txt_enc_BayesianUniSkip_batch_size_256_lr_0.0001_lr_decay_rate_0.25_hidden_list_[1600, 1600]_attention_fusion_type_block_final_fusion_type_block/best_model.pth


100%|██████████| 3350/3350 [10:26<00:00,  5.35it/s]


Saving results to /auto/homes/bat34/VQA_PartII/baseline/attention_block_val-2015_results.json
Done saving to /auto/homes/bat34/VQA_PartII/baseline/attention_block_val-2015_results.json


In [ ]:
aggs = ['mean', 'max', 'min', 'sum']
q_atts = ['True', 'False']

for agg in aggs:
    for q_att in q_atts:
        if q_att == 'True':
            config['q_self_attention'] = True
            config['fusion_mlp']['input_dims'][0] = 4800
        else:
            config['q_self_attention'] = False
            config['fusion_mlp']['input_dims'][0] = 2400
        model = AggConcatNet(config, word_vocabulary)
        BEST_MODEL_NAME = BEST_MODEL_NAME_TEMPL.format(agg, agg, q_att)
        BEST_MODEL_PATH = os.path.join(ROOT_DIR, MODEL_NAME, 'trained_models', 'best_models', \
                             BEST_MODEL_NAME, 'best_model.pth')
        RESULTS_FILE_PATH = os.path.join(ROOT_DIR, MODEL_NAME, 'agg_{}_q_att_{}_test-2015_results.json'.format(agg, q_att))
        eval_model(model, BEST_MODEL_PATH, RESULTS_FILE_PATH)

Processing agg_concat_mean_agg_type_mean_q_self_attention_True_txt_enc_BayesianUniSkip_batch_size_256_lr_0.0001_fusion_type_concat_mlp....


100%|██████████| 6997/6997 [21:27<00:00,  5.44it/s]


Saving results to /auto/homes/bat34/VQA_PartII/baseline/agg_mean_q_att_True_test-2015_results.json
Done saving to /auto/homes/bat34/VQA_PartII/baseline/agg_mean_q_att_True_test-2015_results.json
Processing agg_concat_mean_agg_type_mean_q_self_attention_False_txt_enc_BayesianUniSkip_batch_size_256_lr_0.0001_fusion_type_concat_mlp....


100%|██████████| 6997/6997 [08:21<00:00, 13.95it/s]


Saving results to /auto/homes/bat34/VQA_PartII/baseline/agg_mean_q_att_False_test-2015_results.json
Done saving to /auto/homes/bat34/VQA_PartII/baseline/agg_mean_q_att_False_test-2015_results.json
Processing agg_concat_max_agg_type_max_q_self_attention_True_txt_enc_BayesianUniSkip_batch_size_256_lr_0.0001_fusion_type_concat_mlp....


...

100%|██████████| 6997/6997 [08:00<00:00, 14.56it/s]


Saving results to /auto/homes/bat34/VQA_PartII/baseline/agg_min_q_att_True_test-2015_results.json
Done saving to /auto/homes/bat34/VQA_PartII/baseline/agg_min_q_att_True_test-2015_results.json
Processing agg_concat_min_agg_type_min_q_self_attention_False_txt_enc_BayesianUniSkip_batch_size_256_lr_0.0001_fusion_type_concat_mlp....


100%|██████████| 6997/6997 [08:27<00:00, 13.79it/s]


Saving results to /auto/homes/bat34/VQA_PartII/baseline/agg_min_q_att_False_test-2015_results.json
Done saving to /auto/homes/bat34/VQA_PartII/baseline/agg_min_q_att_False_test-2015_results.json
Processing agg_concat_sum_agg_type_sum_q_self_attention_True_txt_enc_BayesianUniSkip_batch_size_256_lr_0.0001_fusion_type_concat_mlp....


 56%|█████▋    | 3939/6997 [04:30<03:03, 16.66it/s]

In [6]:
config

{'ROOT_DIR': '/auto/homes/bat34/VQA_PartII/baseline/',
 'agg_type': 'min',
 'batch_size': 256,
 'bottom_up_features_dir': '/auto/homes/bat34/2018-04-27_bottom-up-attention_fixed_36/',
 'checkpoint_every': 1,
 'checkpoint_option': 'resume_last',
 'dropout': 0.2,
 'epochs': 25,
 'fusion_block': {'chunks': 20,
  'dropout_input': 0.1,
  'dropout_prelin': 0.0,
  'input_dims': [4800, 2048],
  'mm_dim': 1000,
  'output_dims': 3000,
  'rank': 15,
  'type': 'block'},
 'fusion_mlp': {'dropout': 0.2,
  'hidden_list': [3200, 3200],
  'input_dims': [4800, 2048],
  'out_dim': 3000},
 'fusion_type': 'concat_mlp',
 'gradual_warmup_steps': [1.0, 4.0, 7.0],
 'log_every': 50,
 'lr': 0.0001,
 'lr_decay_epochs': [14, 24, 2],
 'lr_decay_rate': 0.25,
 'name': 'agg_concat_min',
 'num_workers': 4,
 'processed_dir': '/auto/homes/bat34/VQA_PartII/data/processed_splits/',
 'q_att': {'q_linear0': {'input_dim': 2400, 'output_dim': 512},
  'q_linear1': {'input_dim': 512, 'output_dim': 2}},
 'q_self_attention': False

In [ ]:
results = []
with torch.no_grad():
    for data in tqdm.tqdm(test_loader):
        item = {\
            'question_ids': data['question_ids'].cuda(), \
            'object_features_list': data['object_features_list'].cuda(), \
            'bounding_boxes': data['bounding_boxes'].cuda(), \
            'question_lengths': data['question_lengths'].cuda(), \
        }
        inputs = item
        qids = data['question_unique_id']
        outputs = model(inputs)
        values, ans_indices = torch.max(outputs, dim=1)
        ans_indices = list(ans_indices)
        ans_indices = [tsr.item() for tsr in ans_indices]
        for qid, ans_idx in zip(qids, ans_indices):
            results.append({
                'question_id': int(qid),
                'answer': test_dataset.aid_to_ans[ans_idx]
            })

In [ ]:
#RESULTS_FILE_PATH = os.path.join(ROOT_DIR, MODEL_NAME, '{}_test-2015_results.json'.format(BEST_MODEL_NAME))
RESULTS_FILE_PATH = os.path.join(ROOT_DIR, MODEL_NAME, 'baseline_self_attention_block_test-2015_results.json')
with open(RESULTS_FILE_PATH, 'w') as f:
    json.dump(results, f)

In [ ]:
RESULTS_FILE_PATH = os.path.join(ROOT_DIR, MODEL_NAME, '{}_test-2015_results.json'.format(BEST_MODEL_NAME))

In [ ]:
RESULTS